# Gradio Travel Planner

<a target="_blank" href="https://colab.research.google.com/github/dannys0n/CS394-MyModulesRepo/blob/main/notebooks/02/Gradio%20Travel%20Planner.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://github.com/dannys0n/CS394-MyModulesRepo/tree/main/notebooks/02/Gradio Travel Planner.ipynb">
  <img src="https://img.shields.io/badge/Download_.ipynb-blue" alt="Download .ipynb"/>
</a>

## Install required packages

In [ ]:
!uv pip install gradio==5.49.1 openai

## Set the OpenRouter API Key from Colab Secrets + initialize client

In [ ]:
from google.colab import userdata
import openai

OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')

# Initialize OpenAI client
client = openai.OpenAI(
    base_url='https://openrouter.ai/api/v1',
    api_key=OPENROUTER_API_KEY,
)

## Define some datastructures for prompting

I chose these prompts to steer the models away from asking follow-up questions, assisting the user to structured output after streaming instead. my mileage varied.

gpt and mistral were chosen as two opposite ends to test the quality of output and my system prompts. qwen was originally the only small model, but I found it took a while to start generating. my assumption is it's a popular model in OpenRouter and was in a queue

In [ ]:
from pydantic import BaseModel
from typing import List

# system prompts to pass
CHAT_SYSTEM_PROMPT = {
    "role": "system",
    "content": "You are a travel planning expert AI assistant. "
               "How can you help the user solve their travel question? "
               "Do not ask follow-up questions, assume a structured output model will take over right after your completion based on your output."
}
STRUCTURED_SYSTEM_PROMPT = {
    "role": "system",
    "content": "From your thinking, create an itinerary. Take the following previous output and make it structured in json"
}

# models for user to pick in UI
AVAILABLE_MODELS = [
    "mistralai/ministral-8b-2512",
    "qwen/qwen3-14b",
    "openai/gpt-5.2-chat",
]

# data structures for an Itinerary
class DayPlan(BaseModel):
    day: int
    summary: str
    activities: List[str]

class Itinerary(BaseModel):
    destination: str
    budget: int
    daily_schedule: List[DayPlan]
    notes: str



## Define functions for streaming and structured output

Ideally, chat_with_streaming wouldn't implicitly call other functions, I was just lazy.

In [ ]:
# stream output (prompts for structured output + export)
def chat_with_streaming(message, history, model_name):
    if not history:
        messages = [CHAT_SYSTEM_PROMPT, {"role": "user", "content": message}]
    else:
        messages = history + [{"role": "user", "content": message}]

    stream = client.chat.completions.create(
        model=model_name,
        messages=messages,
        stream=True
    )

    response_text = ""

    # streaming back response
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            token = chunk.choices[0].delta.content
            response_text += token
            yield response_text, None, None

    # stream finished
    # generate structured output
    structured = generate_structured_output(response_text, model_name)
    # generate export
    file_path = export_itinerary(structured)

    # final update (chat + JSON + export)
    yield response_text, structured, file_path

# prompt model for structured output
def generate_structured_output(response_text, model_name):
    messages = [
        STRUCTURED_SYSTEM_PROMPT,
        {
            "role": "user",
            "content": response_text
        }
    ]

    response = client.chat.completions.parse(
        model=model_name,
        messages=messages,
        response_format=Itinerary
    )
    return response.choices[0].message.content


## Define functions for export structured output

Exports pure json from structured output. If I had more time, markdown to pdf would have been my choice of prettying export

In [ ]:
import json
import tempfile
from pathlib import Path

def export_itinerary(itinerary_json: str) -> str:
    tmp = tempfile.NamedTemporaryFile(
        mode="w",
        suffix=".json",
        delete=False,
        encoding="utf-8"
    )

    data = json.loads(itinerary_json)
    json.dump(data, tmp, indent=2, ensure_ascii=False)
    tmp.close()

    return tmp.name


## Start chat interface

UI showing streaming, structured output, and download export

In [ ]:
import gradio as gr

# Create chat interface
with gr.Blocks() as demo:
    gr.Markdown("## Travel Planner AI")

    # dropdown to select model
    model_dropdown = gr.Dropdown(
        choices=AVAILABLE_MODELS,
        value=AVAILABLE_MODELS[0],
        label="Model"
    )

    # grouping outputs
    with gr.Row():
          # defining structured output + download
          with gr.Column(scale=1):
              structured_json = gr.JSON(scale=3,label="Structured Output")

              download_file = gr.File(
                  label="Download Itinerary",
                  file_types=[".json"],
              )
          # overall chat window
          with gr.Column(scale=3):
              chat = gr.ChatInterface(
                  fn=chat_with_streaming, # call function and pass arguments
                  additional_inputs=[model_dropdown],
                  type="messages",
                  additional_outputs=[
                      structured_json,
                      download_file
                  ]
              )


demo.launch(share=True, debug=True)



## Use of AI Tools (Academic Disclosure)

AI-assisted tools (ChatGPT-5.2 Auto) were used during the development of this notebook to:
- Explore design options for a Gradio-based chat interface
- Assist in refining Python code
- Debug runtime and UI issues

All generated content was reviewed, modified, and validated by the author.
The author retains responsibility for the correctness and originality of the final submission.